# 🏭 Model3 Greenhouse 데이터셋 전처리

**목적**: 임의로 분할된 train/val/test 데이터를 과학적으로 재구성

**주요 기능**:
- ✅ 계층화 분할 (Stratified Split)
- ✅ 클래스 균형 유지
- ✅ YOLO 형식 디렉토리 구조
- ✅ 재현성 보장 (random_seed=42)

**작성**: Claude Sonnet  
**날짜**: 2025-11-04

In [14]:
# 필수 라이브러리 임포트
import os
import shutil
import random
import yaml
from pathlib import Path
from typing import Dict, List, Tuple
from collections import Counter, defaultdict
from dataclasses import dataclass

# tqdm 임포트 (노트북 버전 시도, 실패 시 일반 버전 사용)
try:
    from tqdm.notebook import tqdm
    print("✅ tqdm.notebook 사용")
except ImportError:
    from tqdm import tqdm
    print("✅ tqdm 사용 (진행바는 텍스트로 표시됩니다)")

print("✅ 라이브러리 임포트 완료")

✅ tqdm.notebook 사용
✅ 라이브러리 임포트 완료


In [15]:
# 설정
SOURCE_DIR = r"C:\Users\LX\Nong-View\model3_greenhouse"
OUTPUT_DIR = r"C:\Users\LX\Nong-View\model3_greenhouse_processed"

TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
RANDOM_SEED = 42

# 클래스 이름
CLASS_NAMES = {0: 'Greenhouse_single', 1: 'Greenhouse_multi'}

# 랜덤 시드 설정
random.seed(RANDOM_SEED)

print(f"📁 소스 디렉토리: {SOURCE_DIR}")
print(f"📁 출력 디렉토리: {OUTPUT_DIR}")
print(f"📊 분할 비율: Train={TRAIN_RATIO}, Val={VAL_RATIO}, Test={TEST_RATIO}")
print(f"🎲 랜덤 시드: {RANDOM_SEED}")

📁 소스 디렉토리: C:\Users\LX\Nong-View\model3_greenhouse
📁 출력 디렉토리: C:\Users\LX\Nong-View\model3_greenhouse_processed
📊 분할 비율: Train=0.8, Val=0.1, Test=0.1
🎲 랜덤 시드: 42


In [16]:
# 이미지 정보 클래스
@dataclass
class ImageInfo:
    image_path: Path
    label_path: Path
    filename: str
    classes: List[int]
    dominant_class: int

## 1️⃣ 데이터 수집
기존 train/val/test 폴더의 모든 이미지와 라벨을 수집합니다.

In [17]:
def parse_label_file(label_path: Path) -> List[int]:
    """YOLO 형식 라벨 파일 파싱"""
    classes = []
    try:
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    classes.append(class_id)
    except Exception as e:
        print(f"⚠ 라벨 파싱 실패: {label_path.name} - {e}")
    return classes


def collect_all_images(source_dir: Path) -> List[ImageInfo]:
    """모든 이미지 수집"""
    all_images = []
    
    for split in ['train', 'val', 'test']:
        images_dir = source_dir / 'images' / split
        labels_dir = source_dir / 'labels' / split
        
        if not images_dir.exists():
            print(f"⚠ 디렉토리 없음: {images_dir}")
            continue
        
        # 이미지 파일 찾기
        image_files = list(images_dir.glob('*.png')) + list(images_dir.glob('*.jpg'))
        
        for img_path in image_files:
            label_path = labels_dir / f"{img_path.stem}.txt"
            
            if label_path.exists():
                classes = parse_label_file(label_path)
                
                if classes:
                    # 가장 많은 객체를 가진 클래스
                    class_counts = Counter(classes)
                    dominant_class = class_counts.most_common(1)[0][0]
                    
                    all_images.append(ImageInfo(
                        image_path=img_path,
                        label_path=label_path,
                        filename=img_path.name,
                        classes=classes,
                        dominant_class=dominant_class
                    ))
    
    return all_images

In [18]:
# 데이터 수집 실행
print("🔍 데이터 수집 중...")
all_images = collect_all_images(Path(SOURCE_DIR))
print(f"✅ 총 {len(all_images)}개 이미지 수집 완료")

🔍 데이터 수집 중...
✅ 총 1483개 이미지 수집 완료


## 2️⃣ 데이터 분석
수집된 데이터의 통계를 분석합니다.

In [19]:
# 통계 분석
total_objects = 0
class_distribution = Counter()

for img_info in all_images:
    total_objects += len(img_info.classes)
    for class_id in img_info.classes:
        class_distribution[class_id] += 1

# 결과 출력
print("\n📊 데이터 통계:")
print(f"  - 총 이미지: {len(all_images)}개")
print(f"  - 총 객체: {total_objects}개")
print(f"  - 클래스 분포:")

for class_id, count in sorted(class_distribution.items()):
    class_name = CLASS_NAMES.get(class_id, f'Class_{class_id}')
    percentage = (count / total_objects) * 100
    print(f"    • {class_name}: {count}개 ({percentage:.1f}%)")


📊 데이터 통계:
  - 총 이미지: 1483개
  - 총 객체: 2567개
  - 클래스 분포:
    • Greenhouse_single: 1602개 (62.4%)
    • Greenhouse_multi: 965개 (37.6%)


## 3️⃣ 계층화 분할 (Stratified Split)
클래스 비율을 유지하면서 train/val/test로 분할합니다.

In [20]:
def stratified_split(images: List[ImageInfo], 
                    train_ratio: float, val_ratio: float, test_ratio: float,
                    random_seed: int = 42) -> Tuple[List[ImageInfo], List[ImageInfo], List[ImageInfo]]:
    """계층화 분할"""
    # 클래스별로 이미지 그룹화
    class_groups = defaultdict(list)
    for img_info in images:
        class_groups[img_info.dominant_class].append(img_info)
    
    train_images = []
    val_images = []
    test_images = []
    
    # 각 클래스별로 분할
    for class_id, class_images in class_groups.items():
        # 셔플
        random.shuffle(class_images)
        
        n_total = len(class_images)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)
        
        # 분할
        train_images.extend(class_images[:n_train])
        val_images.extend(class_images[n_train:n_train + n_val])
        test_images.extend(class_images[n_train + n_val:])
        
        class_name = CLASS_NAMES.get(class_id, f'Class_{class_id}')
        print(f"  • {class_name}: Train={n_train}, Val={n_val}, Test={n_total - n_train - n_val}")
    
    # 최종 셔플
    random.shuffle(train_images)
    random.shuffle(val_images)
    random.shuffle(test_images)
    
    return train_images, val_images, test_images

In [21]:
# 계층화 분할 실행
print("\n🔀 계층화 분할 중...")
train_images, val_images, test_images = stratified_split(
    all_images, TRAIN_RATIO, VAL_RATIO, TEST_RATIO, RANDOM_SEED
)

print(f"\n✅ 분할 완료:")
print(f"  - Train: {len(train_images)}개")
print(f"  - Val: {len(val_images)}개")
print(f"  - Test: {len(test_images)}개")


🔀 계층화 분할 중...
  • Greenhouse_multi: Train=607, Val=75, Test=77
  • Greenhouse_single: Train=579, Val=72, Test=73

✅ 분할 완료:
  - Train: 1186개
  - Val: 147개
  - Test: 150개


## 4️⃣ 출력 디렉토리 생성
YOLO 형식의 디렉토리 구조를 생성합니다.

In [22]:
# 출력 디렉토리 생성
output_path = Path(OUTPUT_DIR)

# 기존 디렉토리 삭제 (주의: 기존 데이터가 있으면 삭제됩니다!)
if output_path.exists():
    print(f"⚠ 기존 디렉토리 삭제: {output_path}")
    shutil.rmtree(output_path)

# YOLO 형식 디렉토리 구조 생성
for split in ['train', 'val', 'test']:
    (output_path / 'images' / split).mkdir(parents=True, exist_ok=True)
    (output_path / 'labels' / split).mkdir(parents=True, exist_ok=True)

print(f"✅ 출력 디렉토리 생성 완료: {output_path}")

⚠ 기존 디렉토리 삭제: C:\Users\LX\Nong-View\model3_greenhouse_processed
✅ 출력 디렉토리 생성 완료: C:\Users\LX\Nong-View\model3_greenhouse_processed


## 5️⃣ 데이터 복사
분할된 데이터를 새로운 디렉토리 구조로 복사합니다.

In [23]:
def copy_split_data(split_name: str, images: List[ImageInfo], output_path: Path):
    """분할된 데이터 복사"""
    images_dir = output_path / 'images' / split_name
    labels_dir = output_path / 'labels' / split_name
    
    class_dist = Counter()
    
    for img_info in tqdm(images, desc=f"{split_name} 복사"):
        # 이미지 복사
        shutil.copy2(img_info.image_path, images_dir / img_info.filename)
        
        # 라벨 복사
        shutil.copy2(img_info.label_path, labels_dir / f"{Path(img_info.filename).stem}.txt")
        
        # 클래스 분포 업데이트
        for class_id in img_info.classes:
            class_dist[class_id] += 1
    
    return class_dist

In [24]:
# 데이터 복사 실행
print("\n📋 데이터 복사 중...\n")

split_stats = {}
for split_name, split_images in [('train', train_images), ('val', val_images), ('test', test_images)]:
    class_dist = copy_split_data(split_name, split_images, output_path)
    split_stats[split_name] = {
        'images': len(split_images),
        'objects': sum(class_dist.values()),
        'class_distribution': dict(class_dist)
    }

print("\n✅ 데이터 복사 완료")


📋 데이터 복사 중...



ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

## 6️⃣ YAML 설정 파일 생성
YOLO 훈련을 위한 data.yaml 파일을 생성합니다.

In [ ]:
# YAML 파일 생성
yaml_content = {
    'path': str(output_path.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 2,
    'names': ['Greenhouse_single', 'Greenhouse_multi'],
    
    'dataset_info': {
        'total': len(all_images),
        'train': len(train_images),
        'val': len(val_images),
        'test': len(test_images),
        'augmented': 0
    },
    
    'preprocessing': {
        'method': 'stratified_split',
        'train_ratio': TRAIN_RATIO,
        'val_ratio': VAL_RATIO,
        'test_ratio': TEST_RATIO,
        'random_seed': RANDOM_SEED
    }
}

yaml_path = output_path / 'data.yaml'
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(yaml_content, f, default_flow_style=False, allow_unicode=True, sort_keys=False)

print(f"✅ YAML 파일 생성: {yaml_path}")

## 7️⃣ 최종 요약
재구성된 데이터셋의 통계를 출력합니다.

In [ ]:
# 최종 요약 출력
print("\n" + "=" * 60)
print("✨ 데이터셋 재구성 완료!")
print("=" * 60)

print("\n📊 최종 데이터 분포:")

for split_name in ['train', 'val', 'test']:
    split_info = split_stats[split_name]
    print(f"\n  📁 {split_name.upper()}:")
    print(f"    - 이미지: {split_info['images']}개")
    print(f"    - 객체: {split_info['objects']}개")
    print(f"    - 클래스 분포:")
    
    for class_id, count in sorted(split_info['class_distribution'].items()):
        class_name = CLASS_NAMES.get(class_id, f'Class_{class_id}')
        percentage = (count / split_info['objects']) * 100
        print(f"      • {class_name}: {count}개 ({percentage:.1f}%)")

print(f"\n🎯 재구성된 데이터셋 위치: {output_path}")
print(f"🎯 설정 파일: {yaml_path}")
print("\n✅ 이제 YOLO 훈련을 시작할 수 있습니다!")

In [ ]:
# 추가: 분할 비율 검증
print("\n📈 분할 비율 검증:")
total = len(all_images)
print(f"  - Train: {len(train_images)/total*100:.1f}% (목표: {TRAIN_RATIO*100}%)")
print(f"  - Val: {len(val_images)/total*100:.1f}% (목표: {VAL_RATIO*100}%)")
print(f"  - Test: {len(test_images)/total*100:.1f}% (목표: {TEST_RATIO*100}%)")